In [ ]:
# import cell

import numpy as np
from numpy import linalg as LA

## 算子

- `matmul` : Matrix product of two arrays. '@' operator as method with out parameter.
- `dot` : Dot product of two arrays. Dot multiplication with two arguments.
- `vdot` : Return the dot product of two vectors. Complex-conjugating dot product.
- `linalg.multi_dot` : Chained dot product. Compute the dot product of two or more arrays in a single function call.
- `tensordot` : Compute tensor dot product along specified axes. Sum products over arbitrary axes.
- 
- `inner` : Inner product of two arrays.
- `outer` : Compute the outer product of two vectors.
- `einsum` : Einstein summation convention. Evaluates the Einstein summation convention on the operands.

[内积、点积、数量积有何区别？](https://www.zhihu.com/question/296585022)  
[difference between dot product and inner product](https://www.zhihu.com/question/296585022)  
[20+ examples for NumPy matrix multiplication](https://likegeeks.com/numpy-matrix-multiplication/)  

## matmul

The behavior depends on the arguments in the following way.

1. If the first argument is 1-D, it is promoted to a matrix by prepending a 1 to its dimensions. After matrix multiplication the prepended 1 is removed.
    - a[np.newaxis, :] 提升为2-D数组（行）。
2. If the second argument is 1-D, it is promoted to a matrix by appending a 1 to its dimensions. After matrix multiplication the appended 1 is removed.
    - b[:, np.newaxis] 提升为2-D数组（列）。
3. If both arguments are 2-D they are multiplied like conventional matrices.
    - 通常意义上的矩阵乘法。
4. If either argument is N-D, N > 2, it is treated as a stack of matrices residing in the last two indexes and broadcast accordingly.
    - 把最后两维作为矩阵对待。

The `numpy.matmul` function implements the `@` operator.

- The matmul function implements the semantics of the `@` operator introduced in Python 3.5 following [PEP 465](https://peps.python.org/pep-0465/).
- Introduced in NumPy 1.10.0, the `@` operator is *preferable* to other methods when computing the matrix product between 2d arrays.

In [ ]:
# matmul samples

a=np.arange(1,3)
b=np.arange(1,5).reshape(2,2)
c1 = a @ b # 等效于 a[np.newaxis, :] @ b

a=np.arange(1,5).reshape(2,2)
b=np.arange(1,3)
c2 = a @ b # 等效于 a @ b[:, np.newaxis]

# 2D @ 2D
a=np.arange(1,5).reshape(2,2)
b=np.arange(5,9).reshape(2,2)
c3 = a @ b # 矩阵乘法

### 3D @ 3D

[Numpy-3d Matrix Multiplication](https://www.javatpoint.com/numpy-3d-matrix-multiplication)

- [3d Matrix multiplication in numpy](https://stackoverflow.com/questions/65583672/3d-matrix-multiplication-in-numpy)
- [Matrix multiplication of inner dimensions of 3D tensors?](https://stackoverflow.com/questions/42935288/matrix-multiplication-of-inner-dimensions-of-3d-tensors)

If either argument is N-D, N > 2, it is treated as a **stack** of matrices residing in the *last two* indexes and broadcast accordingly.

对于 3D @ 3D，把最后两维作为stack矩阵对待。

$ x(2,2,4) = \left[ \begin{array}{cccc|cccc} 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\ \hline 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 \\ \end{array} \right] $ = $ \begin{bmatrix} x_{00} & x_{01} \\ x_{10} & x_{11} \\ \end{bmatrix} $，每一块 $x_i$ 代表第三维长度为4的向量。

$ y(2,4,2) = \left[ \begin{array}{cc|cc|cc|cc} 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\ \hline 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 \\ \end{array} \right] $ = $ \begin{bmatrix} y_{00} & y_{01} & y_{02} & y_{03} \\ y_{10} & y_{11} & y_{12} & y_{13} \\ \end{bmatrix} $，每一块 $y_i$ 代表第三维长度为2的向量。

x @ y：x的后两维(2,4)和y的后两维(4,2)得到(2,2)，故结果shape=(2,2,2)。

$ x[0] = \begin{bmatrix} 0 & 1 & 2 & 3 \\ 4 & 5 & 6 & 7 \\ \end{bmatrix} $，
$ y[0] = \begin{bmatrix} 0 & 1 \\ 2 & 3 \\ 4 & 5 \\ 6 & 7 \\ \end{bmatrix} $，$ z[0] = x[0] @ y[0] $


$ x[1] = \begin{bmatrix} 8 & 9 & 10 & 11 \\ 12 & 13 & 14 & 15 \\ \end{bmatrix} $，
$ y[1] = \begin{bmatrix} 8 & 9 \\ 10 & 11 \\ 12 & 13 \\ 14 & 15 \\ \end{bmatrix} $，$ z[1] = x[1] @ y[1] $

z = x @ y 的等效算法如下：

```Python
# z[i] 只指定第一维，表示其他两维矩阵
for i in range(len(x)):
    z[i] = x[i] @ y[i]
```

In [ ]:
# 3D @ 3D

x = np.arange(2 * 2 * 4).reshape((2, 2, 4))
y = np.arange(2 * 2 * 4).reshape((2, 4, 2))
z = x @ y
print('z:\n', z)

# 等效的其他写法：
z = np.einsum('ijk,ikl->ijl', x, y)
z = [np.matmul(a, b) for a, b in zip(x, y)]
# 列表推导的for循环展开
z = np.zeros((2,2,2), dtype=np.int32)
for i in range(len(x)): # x.shape[0]
    z[i] = x[i] @ y[i]


## inner

`inner(a, b, /)`: Inner product of two arrays.

Ordinary inner product of vectors for 1-D arrays (without complex conjugation), in higher dimensions a sum product over the **last** axes.

For vectors (1-D arrays) it computes the ordinary inner-product:

> 对于1D数组（向量），就是普通的内积。

```Python
np.inner(a, b) = sum(a[:]*b[:])
```

More generally, if ndim(a) = r > 0 and ndim(b) = s > 0:

> 对于维度大于等于2的数组，axes 取各自最后一维进行 *tensordot。*

```Python
np.inner(a, b) = np.tensordot(a, b, axes=(-1,-1))
```

In [ ]:
# inner

a = np.arange(24).reshape((2,3,4))
b = np.arange(4)
# a的第三维·b的第一维
c1 = np.inner(a, b) # 等价于 a @ b

a = np.arange(2).reshape((1,1,2))
b = np.arange(6).reshape((3,2))
# a的第三维·b的第二维
c2 = np.inner(a, b) 
# a @ b 报错 size 3 is different from 2

## outer

`outer(a, b, out=None)`: Compute the outer product of two vectors.

> out[i, j] = a[i] * b[j]


In [ ]:
# outer

rl = np.outer(np.ones((5,)), np.linspace(-2, 2, 5))
im = np.outer(1j*np.linspace(2, -2, 5), np.ones((5,)))
x = np.array(['a', 'b', 'c'], dtype=object)
np.outer(x, [1, 2, 3])

## dot

`dot(a, b, out=None)`: Dot product of two arrays.

**Parameters**:

- a : array_like
- b : array_like
- out : ndarray, optional

**Specifically**:

- If both `a` and `b` are 1-D arrays, it is inner product of vectors (without complex conjugation).

- If both `a` and `b` are 2-D arrays, it is matrix multiplication, but using :func:`matmul` or ``a @ b`` is preferred.

- If either `a` or `b` is 0-D (scalar), it is equivalent to :func:`multiply` and using ``numpy.multiply(a, b)`` or ``a * b`` is preferred.

- If `a` is an N-D array and `b` is a 1-D array, it is a <u>sum product</u> over the *last* axis of `a` and `b`.

- If `a` is an N-D array and `b` is an M-D array (where ``M>=2``), it is a <u>sum product</u> over the *last* axis of `a` and the *second-to-last* axis of `b`::

    dot(a, b)[i,j,k,m] = sum(a[i,j,:] * b[k,:,m])

### matrix notation

The dot product $x{\cdot}y$ is a row vector times a column vector.

We know the dot product (inner product) of $x$ and $y$. It is the sum of numbers $ x_iy_i $（$ \sum_{i=1}^n x_iy_i $）  
Now we have a better way to write $x{\cdot}y$, without using that unprofessional dot. Use matrix notation instead:

- $^T$ is inside: The dot product or inner product is $x^Ty$ -- (1xn)(nx1)
- $^T$ is outside: The rank one product or outer product is $xy^T$ -- (nx1)(1xn)

$x^Ty$ is a number, $xy^T$ is a matrix.  
Quantum mechanics would write those as $ <x|y> $ (inner) and $ |x><y| $ (outer).  
Maybe the universe is governed by linear algebra.

对于1D数组，dot点积相当于inner内积：

$ x = \begin{bmatrix} x_1 \\ x_2 \\ \end{bmatrix} $ and $ y = \begin{bmatrix} y_1 \\ y_2 \\ \end{bmatrix} $ then $x{\cdot}y$ = $x_1y_1 + x_2y_2$ = $ \sum_{i=1}^n x_iy_i $

$x{\cdot}y = x^Ty = \begin{bmatrix} x_1 & x_2 \\ \end{bmatrix} \begin{bmatrix} y_1 \\ y_2 \\ \end{bmatrix} = x_1y_1 + x_2y_2 $

$ xy^T = \begin{bmatrix} x_1 \\ x_2 \\ \end{bmatrix} \begin{bmatrix} y_1 & y_2 \\ \end{bmatrix} = \begin{bmatrix} x_1y_1 & x_1y_2 \\ x_2y_1 & x_2y_2 \\ \end{bmatrix} $

### 1-D vector

If both a and b are 1-D arrays, it is `inner` product of vectors (without complex conjugation).

一维数组的点积，自动转换成内积，也可采用标准的矩阵乘法算子 `matmul(@)`。

> 计算结果为数值标量。

In [ ]:
# 1-D list dot product

# 标量建议采用 multiply(*) 算子
s1 = np.dot(3, 4)
print('s1 =', s1)

# np.dot 和 np.matmul 支持传入两个1维列表，内部会自动转换为ndarray。
l1 = [1,2]
l2 = [4,5]

dotl1 = np.dot(l1,l2)
print('dotl1 =', dotl1)

dotl2 = np.matmul(l1, l2)
print('dotl2 =', dotl2)

In [ ]:
# 1-D array dot product

a1=np.array([1,2])
a2=np.array([4,5])

dot1 = np.dot(a1, a2) # or a1.dot(a2)
print('dot1 =', dot1)

# @ 等效于 np.matmul(a1, a2)
dot2 = a1 @ a2
print('dot2 =', dot2)

# perpendicular: a3 @ a4 = 0
a3 = np.array([1,3,2])
a4 = np.array([4,-4,4])
dot3 = a3 @ a4
print('dot3 =', dot3)

### 2-D array

If both a and b are 2-D arrays, it is matrix multiplication, but using `matmul` or `a @ b` is preferred.

二维数组（矩阵）的点积（内积）需要符合矩阵的乘法阶数要求，建议直接采用 `matmul(@)` 算子。

> 计算结果为1x1阶矩阵。

In [ ]:
# 2-D array dot product(inner product)

# two standard column vector represent by matrix
v1=np.array([[1,2]]).T
v2=np.array([[4,5]]).T
# 需转换成内积形式，以使阶数匹配符合矩阵乘法的要求，否则报错 shapes not aligned。
dot1 = np.dot(v1.T, v2) # or v1.T.dot(v2)
print('dot1:\n', dot1)
# @ 等效于 np.matmul(v1.T, v2)
dot2 = v1.T @ v2 # 推荐
print('dot2:\n', dot2)

# np.dot 和 np.matmul 支持传入两个1维列表，内部会自动转换为ndarray。
l21 = [[1, 0], [0, 1]]
l22 = [[4, 1], [2, 2]]
dot3 = np.dot(l21, l22)
print('dot3:\n', dot3)
dot4 = np.matmul(l21, l22)
print('dot4:\n', dot4)
# 二维列表要转换成2-D ndarray，才支持@运算符。
m1 = np.array(l21)
m2 = np.array(l22)
dot5 = m1 @ m2
print('dot5:\n', dot5)

### N-D · 1-D

If a is an N-D array and b is a 1-D array, it is a sum product over the last axis of a and b.

若 $ a = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix}, b = \begin{bmatrix} 5 & 6 \\ \end{bmatrix} $ 则 $a{\cdot}b = ab^T = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} \begin{bmatrix} 5 \\ 6 \\ \end{bmatrix} $

相当于 ((a.row1)·(b.row), (a.row2)·(b.row))。

In [ ]:
# 2-D · 1-D

l1 = [[1, 2], [3, 4]]
l2 = [5, 6]
a=np.array(l1)
b=np.array(l2)
dot_n1 = np.dot(a, b)
print('dot_n1:\n', dot_n1)

# 1-D · 2-D

c=np.arange(3)
d=np.arange(3*2).reshape(3,2)
dot_1n = np.dot(c, d)
print('dot_1n:\n', dot_1n)


### N-D · M-D

If a is an N-D array and b is an M-D array (where M>=2), it is a **sum product** over the *last* axis of a and the *second-to-last* axis of b.


#### 2D · 3D

shape=(2,2) 的2-D数组 n_22 = $ \begin{bmatrix} 0 & 1 \\ 2 & 3 \\ \end{bmatrix} $ = $ \begin{bmatrix} n_{00} & n_{01} \\ n_{10} & n_{11} \\ \end{bmatrix} $  
shape=(2,2,3) 的D-D数组 m_223 如下（将其划分为2x2的方块 $m_{00}$,$m_{01}$,$m_{10}$,$m_{11}$）：

m_223 = $ \left[ \begin{array}{ccc|ccc} 0 & 1 & 2 & 3 & 4 & 5 \\ \hline 6 & 7 & 8 & 9 & 10 & 11 \end{array} \right] $ = $ \begin{bmatrix} m_{00} & m_{01} \\ m_{10} & m_{11} \\ \end{bmatrix} $，每一块 $m_i$ 代表第三维长度为3的向量。

则 $n{\cdot}m$ = $ \begin{bmatrix} n_{00} & n_{01} \\ n_{10} & n_{11} \\ \end{bmatrix} \begin{bmatrix} m_{00} & m_{01} \\ m_{10} & m_{11} \\ \end{bmatrix} $ = $ \begin{bmatrix} n_{00}m_{00}+n_{01}m_{01} & n_{00}m_{10}+n_{01}m_{11} \\ n_{10}m_{00}+n_{11}m_{01} & n_{10}m_{10}+n_{11}m_{11} \\ \end{bmatrix} $

此时，n的倒数第一个axis=1，m的倒数第二个axis=1，故 `np.dot(n,m)` 等效于 `np.tensordot(n, m, axes=(1,1))`。

In [ ]:
n_22 = np.arange(4).reshape(2,2)
m_223 = np.arange(12).reshape(2,2,3)
print('n:\n', n_22)
print('m:\n', m_223)
dot_23 = np.dot(n_22, m_223)
print('dot_23.shape =', dot_23.shape)
print('dot_23:\n', dot_23)
# 等效 tensordot 表达式
tdot_23 = np.tensordot(n_22, m_223, axes=(1,1))
print('tdot_23:\n', tdot_23)

#### 3D · 2D

shape=(2,2,3) 的D-D数组 m_223 如下（将其划分为2x2的方块 $m_{00}$,$m_{01}$,$m_{10}$,$m_{11}$）：

m_223 = $ \left[ \begin{array}{ccc|ccc} 0 & 1 & 2 & 3 & 4 & 5 \\ \hline 6 & 7 & 8 & 9 & 10 & 11 \end{array} \right] $ = $ \begin{bmatrix} m_{00} & m_{01} \\ m_{10} & m_{11} \\ \end{bmatrix} $，每一块 $m_i$ 代表第三维长度为3的向量。

shape=(2,2) 的2-D数组 n_32 = $ \begin{bmatrix} 0 & 1 \\ 2 & 3 \\ 4 & 5 \\ \end{bmatrix} $ = $ \begin{bmatrix} n_{00} & n_{01} \\ n_{10} & n_{11} \\ n_{20} & n_{21} \\ \end{bmatrix} $  

则 $m{\cdot}n$ = $ \left[ \begin{array}{cc|cc} m_{00}{\cdot}col1 & m_{00}{\cdot}col2 & m_{01}{\cdot}col1 & m_{01}{\cdot}col2 \\ \hline m_{10}{\cdot}col1 & m_{10}{\cdot}col2 & m_{11}{\cdot}col1 & m_{11}{\cdot}col2 \\ \end{array} \right] $

此时，m的倒数第一个axis=2，n的倒数第二个axis=0，故 `np.dot(m,n)` 等效于 `np.tensordot(m, n, axes=(2,0))`。

In [ ]:
m_223 = np.arange(12).reshape(2,2,3)
n_32 = np.arange(3*2).reshape(3,2)
print('n:\n', n_32)
print('m:\n', m_223)
dot_32 = np.dot(m_223, n_32)
print('dot_32.shape =', dot_32.shape)
print('dot_32:\n', dot_32)
# 等效 tensordot 表达式
tdot_32 = np.tensordot(m_223, n_32, axes=(2,0))
print('tdot_32:\n', tdot_32)

## vdot

`vdot(a, b)`: Return the dot product of two vectors.

The vdot(`a`, `b`) function handles complex numbers differently than dot(`a`, `b`).

- If the first argument is complex the complex conjugate of the first argument is used for the calculation of the dot product.

Note that `vdot` handles multidimensional arrays differently than `dot`: 

- it does *not* perform a matrix product, but flattens input arguments to 1-D vectors first. Consequently, it should only be used for vectors.

In [ ]:
# complex

# 对于复数运算，vdot 将第一个向量中的复数替换成共轭再计算。

np.dot([2j, 3j], [2j, 3j]) # (-13+0j)
# 等效于 np.dot([-2j, -3j], [2j, 3j])
np.vdot([2j, 3j], [2j, 3j]) # (13+0j)

a = np.array([1+2j,3+4j])
b = np.array([5+6j,7+8j])
c = np.dot(a, b)
print('c:\n', c)
d = np.vdot(a, b)
print('d:\n', d)

# 等效于a的共轭与b的点积
ac = np.array([1-2j,3-4j])
e = np.dot(ac, b)
print('d:\n', e)

In [ ]:
# ndarray

# Note that higher-dimensional arrays are flattened!

a = np.array([[1, 4], [5, 6]])
b = np.array([[4, 1], [2, 2]])
c = np.vdot(a, b)
print('c =', c)
# 等效于 np.dot(a.flatten(), b.flatten())
# 1*4 + 4*1 + 5*2 + 6*2
d = np.dot(a.flatten(), b.flatten())
print('d =', d)

## multi_dot

multi_dot **chains** numpy.dot

multi_dot([A, B, C, D]) 等效于：

- np.dot(np.dot(np.dot(A, B), C), D)
- A.dot(B).dot(C).dot(D)

In [ ]:
a = np.arange(60).reshape(3,4,5)
b = np.arange(24).reshape(4,3,2)
c = np.tensordot(a,b, axes=((1,0),(0,1))) # axes=([1,0],[0,1])

# A slower but equivalent way of computing the same...
d = np.zeros((5,2), dtype=np.int32)
for i in range(5):
  for j in range(2):
    print('i, j =', i, j)
    for k in range(3):
      for n in range(4):
        d[i,j] += a[k,n,i] * b[n,k,j]
        print('a[{2},{3},{0}]={4}, b[{3},{2},{1}]={5}, d[{0},{1}]={6}'.format(i,j,k,n,a[k,n,i], b[n,k,j], d[i,j]))